In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, data, color, img_as_ubyte
from skimage.color import rgb2gray
from skimage.morphology import binary_opening, disk
from skimage.transform import rescale, resize
from skimage.feature import canny
from skimage.draw import ellipse_perimeter, line
from skimage.measure import find_contours
from skimage.util import img_as_ubyte
from IPython.utils import traitlets as _traitlets

from scipy import ndimage as ndi
from PIL import Image
from google.colab.patches import cv2_imshow
from google.colab import files

import sys
sys.setrecursionlimit(10000000)

In [0]:
![picture](https://drive.google.com/uc?id=1d9ttrseJh23lVFNTyWIqXn6d85XAYYPh)

/bin/bash: -c: line 0: syntax error near unexpected token `https://drive.google.com/uc?id=1d9ttrseJh23lVFNTyWIqXn6d85XAYYPh'
/bin/bash: -c: line 0: `[picture](https://drive.google.com/uc?id=1d9ttrseJh23lVFNTyWIqXn6d85XAYYPh)'


In [0]:
uploaded = files.upload()

Saving HF-1 formalin - pre - 10x - a - 1.jpg to HF-1 formalin - pre - 10x - a - 1.jpg


In [0]:
class Graph: 
  
    def __init__(self, row, col, g): 
        self.ROW = row 
        self.COL = col 
        self.graph = g 

    def isSafe(self, i, j, visited): 
            # row number is in range, column number 
            # is in range and value is 1  
            # and not yet visited 
            return (i >= 0 and i < self.ROW and 
                    j >= 0 and j < self.COL and 
                    not visited[i][j] and self.graph[i][j]) 
      
        # A utility function to do DFS for a 2D  
        # boolean matrix. It only considers 
        # the 8 neighbours as adjacent vertices 
    def DFS(self, i, j, visited, island): 

        # These arrays are used to get row and  
        # column numbers of 8 neighbours  
        # of a given cell 
        rowNbr = [-1, 0, 1, 0]; 
        colNbr = [0,  -1, 0, 1]; 
          
        # Mark this cell as visited 
        visited[i][j] = True

        # Recur for all connected neighbours 
        for k in range(4): 
            if self.isSafe(i + rowNbr[k], j + colNbr[k], visited): 
                island.append((i + rowNbr[k], j + colNbr[k]))
                self.DFS(i + rowNbr[k], j + colNbr[k], visited, island) 


    def findIslands(self): 
        # Make a bool array to mark visited cells. 
        # Initially all cells are unvisited 
        visited = [[False for j in range(self.COL)]for i in range(self.ROW)] 
        # Initialize count as 0 and travese  
        # through the all cells of 
        # given matrix 
        index = 0
        islands = []
        for i in range(self.ROW): 
            for j in range(self.COL): 
                # If a cell with value 1 is not visited yet,  
                # then new island found 
                if visited[i][j] == False and self.graph[i][j] == 1: 
                    # Visit all cells in this island  
                    # and increment island count
                    island = []
                    self.DFS(i, j, visited, island)
                    islands.append(island) 
                    index += 1
        return islands 

def filter_islands(islands, original_mask, min_size=10):
  new_mask = np.zeros(original_mask.shape)
  for island in islands:
    if len(island) >= min_size: # filters out small tears and holes
      max_x = float("-inf")
      min_x = float("inf")
      max_y = float("-inf")
      min_y = float("inf")
      for x, y in island:
        if x > max_x:
          max_x = x
        if x < min_x:
          min_x = x
        if y > max_y:
          max_y = y
        if y < min_y:
          min_y = y
      r_x = max_x - min_x
      r_y = max_y - min_y

      ellipse_area = 3.14159*r_x*r_y


      if len(island) > 50 and ellipse_area > 0 and len(island) / (ellipse_area) > .125 and (r_x/r_y < 1.75 and r_x/r_y > 0.5): #filters out non-ellipsoid shapes and tears; 20, 2
        keep = True
      # elif  len(island) <= 10 and ellipse_area > 0:
      #   keep = True
      else:
        keep = False
      
      if keep == True:
        for x, y in island:
          new_mask[x][y] = 1

  return new_mask

def nodule_percentage(img):
    count = cv2.countNonZero(img)  # 0 is black -> nonzero is white
    area = 190220.0
    print('Globule area:', count, '  Estimate of Total Area:', area, '  Globule Percentage:', count/area)

def draw_contours(contours, hierarchy, shape):
    ret = np.zeros(shape)
    hull = []
    for i in range(len(contours)):
        hull.append(cv2.convexHull(contours[i], False))
    
    coords = []
    for i in range(len(contours)):
        contour = contours[i]
        contour_area = cv2.contourArea(contour)
        hull_area = cv2.contourArea(hull[i])
        if hull_area - contour_area < .1*contour_area:
            for row in range(len(contour)):
                for col in range(len(contour[row])):
                    x, y = contour[row][col]
                    ret[y][x] = 255
                    coords.append((x,y))
    #cv2_imshow(ret)
    return ret


In [0]:
for file_name in uploaded.keys():
    def get_binary(file_name):
      image = cv2.imread(file_name) 
      print(image.shape)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
      gray = np.where(gray<=100, 0, gray) 
      gray = np.where(gray>=200, 255, gray)
      gray = cv2.GaussianBlur(gray, (3,3), 1)
      _, binary = cv2.threshold(gray, 222, 255, cv2.THRESH_BINARY)
      binary = binary_opening(binary, disk(1))
      return image, gray, binary.astype(np.uint8)

    image, gray, binary = get_binary(file_name)
    np_graph = binary
    graph = np_graph.tolist()
    row = len(graph) 
    col = len(graph[0]) 
  
    g = Graph(row, col, graph) 
    print(g.graph)
    islands = g.findIslands()
    new_mask = filter_islands(islands, np_graph, min_size=1).astype(np.uint8)

    new_mask_g = Graph(row, col, new_mask.tolist())
    new_mask_islands = g.findIslands()

    contours, hierarchy = cv2.findContours(new_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)   
    c = draw_contours(contours, hierarchy, image.shape)
    nodule_percentage(new_mask)

    fig, axes = plt.subplots(1, 6, figsize=(30, 30))
    ax = axes.ravel()
    ax[0].imshow(image)
    ax[0].set_title("Original")
    ax[1].imshow(gray, cmap=plt.cm.gray)
    ax[1].set_title("Grayscale")
    ax[2].imshow(binary, cmap=plt.cm.gray)
    ax[2].set_title("Binary")
    ax[3].imshow(new_mask, cmap=plt.cm.gray)
    ax[3].set_title("New Mask")
    ax[4].imshow(new_mask, cmap=plt.cm.gray)
    ax[4].imshow(image, alpha=0.6)
    ax[4].set_title("Overlay")
    ax[5].imshow(c)
    ax[5].set_title("Contours")
    